In [ ]:
!pip install -U BitsandBytes
!pip install -U accelerate

In [ ]:
import time
from tqdm.notebook import tqdm
import torch
from transformers import AutoTokenizer, LlamaForCausalLM, BitsAndBytesConfig
import pandas as pd
import os
import bitsandbytes
import accelerate

In [ ]:
# Defining paths for dataset and base output directory
data_path = '/content/drive/MyDrive/Colab Notebooks/dataset_150k_with_summaries.csv'
output_csv_path = '/content/drive/MyDrive/Colab Notebooks/summarized_abstracts.csv'
output_base_path = '/content/drive/MyDrive/Colab Notebooks/summarized_abstracts_checkpoint'
model_save_path = '/content/drive/MyDrive/Colab Notebooks/Model/llama-3-8b-bnb-4bit'
checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/checkpoint.txt'


In [ ]:
# Loading the dataset
df = pd.read_csv(data_path)
df = df.iloc[:10000]

In [ ]:
# Initializing or load a checkpoint
start_index = 0
if os.path.exists(checkpoint_path):
    with open(checkpoint_path, 'r') as f:
        start_index = int(f.read().strip())
    print(f"Resuming from checkpoint at index {start_index}")

Resuming from checkpoint at index 4020


In [ ]:
# Loading 4-bit quantization configuration and model
quantization_config = BitsAndBytesConfig(load_in_4bit=True)
model_name = "unsloth/llama-3-8b-bnb-4bit"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(model_name, quantization_config=quantization_config)
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:174: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): Llama

In [ ]:
# Function to generate a summary
def generate_summary_llama(abstract):
    inputs = tokenizer(abstract, return_tensors='pt', truncation=True, max_length=2048)
    with torch.no_grad():
        summary_ids = model.generate(
            inputs['input_ids'],
            max_new_tokens=150,
            min_length=40,
            do_sample=False
        )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


start_time = time.time()

with tqdm(total=len(df), desc="Processing Abstracts") as pbar:
    summaries = []  # Storing summaries for the current batch only
    batch_start_index = start_index  # Tracking the start of each batch

    for index, row in df.iloc[start_index:].iterrows():
        abstract = row['abstract']
        summary = generate_summary_llama(abstract)
        summaries.append(summary)

        # Saving every 20 entries
        if (index + 1) % 20 == 0:
            # Add summaries for the current batch to the DataFrame
            df.loc[batch_start_index:index, 'summary'] = summaries

            # Saving the DataFrame, model, tokenizer, and checkpoint
            df.to_csv(output_csv_path, index=False)
            model.save_pretrained(model_save_path)
            tokenizer.save_pretrained(model_save_path)
            with open(checkpoint_path, 'w') as f:
                f.write(str(index + 1))

            # Resetting for the next batch
            summaries = []
            batch_start_index = index + 1  # Updating the start index for the next batch
            print(f"Checkpoint saved at index {index + 1}")

        # Updating progress bar
        pbar.update(1)

# Saving any remaining summaries and the final checkpoint
if summaries:
    df.loc[batch_start_index:index, 'summary'] = summaries
    df.to_csv(output_csv_path, index=False)
    model.save_pretrained(model_save_path)
    tokenizer.save_pretrained(model_save_path)
    with open(checkpoint_path, 'w') as f:
        f.write(str(index + 1))
    print(f"Final checkpoint saved at index {index + 1}")


total_time = time.time() - start_time
print(f"Total time taken: {total_time} seconds")


Processing Abstracts:   0%|          | 0/10000 [00:00<?, ?it/s]

Checkpoint saved at index 4040


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling f

Checkpoint saved at index 4060


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling f

...

Checkpoint saved at index 8020


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling f

Checkpoint saved at index 8040


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling f

KeyboardInterrupt: 